原文章链接：http://nbviewer.jupyter.org/github/david-cortes/cmfrec/blob/master/example/cmfrec_movielens_sideinfo.ipynb

In [1]:
import pandas as pd, time
from datetime import datetime

In [16]:
ratings=pd.read_csv('/Users/apple/Desktop/Data/New feature vector/merge_transaction.csv',index_col='Unnamed: 0')
ratings.head()

,UserId,ItemId,Rating,Timestamp
0,d8:1d:72:76:a9:85,10101,1.0,2015-09-05 23:54:08
1,cc:29:f5:d1:cc:e7,560606,1.0,2015-09-05 23:48:08
2,00:92:4f:85:9e:15,10101,10.0,2015-09-05 23:43:33
3,80:41:4e:e3:9c:1b,210501,1.0,2015-09-05 23:37:56
4,3c:91:57:37:23:ed,10101,1.0,2015-09-05 23:37:09


ratings是每个mac购买商品的数量，小品类和时间

In [3]:
time_cutoff='2015-08-20'
train=ratings.loc[ratings.Timestamp<=time_cutoff]
test=ratings.loc[ratings.Timestamp>time_cutoff]
users_train=set(list(train.UserId))
items_train=set(list(train.ItemId))
test=test.loc[test.UserId.map(lambda x: x in users_train)]
test=test.loc[test.ItemId.map(lambda x: x in items_train)]
print(train.shape)
print(test.shape)

(13535, 5)
(3644, 5)


定义训练集和测试集

In [4]:
from cmfrec import CMF
import numpy as np
# Number of latent factors
k=40
# Regularization parameter
reg=1e-3
# Fitting the model
rec=CMF(k=k, reg_param=reg)
rec.fit(train, random_seed=12345)
# Making predictions
test['Predicted']=test.apply(lambda x: rec.predict(x['UserId'],x['ItemId']),axis=1)

/Applications/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1.290920
  Number of iterations: 260
  Number of functions evaluations: 296


Evaluating Hold-Out RMSE (the hyperparameters had already been somewhat tuned by cross-validation)

In [5]:
np.sqrt(np.mean((test.Predicted-test.Rating)**2))

1.4502077635508481

adding user side information

Features
genre0: mac平均停留时间
genre1: mac停留时间方差
genre2: mac总出现次数
genre3: mac在周中出现次数
genre4: mac在周末出现次数
genre5: mac的交易次数
genre6: mac最长停留时间
genre7: mac最短停留时间
genre8: mac在时区1（0-6）出现次数
genre9: mac在时区2（6-12）出现次数
genre10: mac在时区3（12-18）出现次数
genre11: mac在时区4（18-24）出现次数
genre12: mac在时区1（0-6）平均停留时间
genre13: mac在时区2（6-12）平均停留时间
genre14: mac在时区3（12-18）平均停留时间
genre15: mac在时区4（18-24）平均停留时间
genre16: mac在时区1（0-6）停留时间方差
genre17: mac在时区2（6-12）停留时间方差
genre18: mac在时区3（12-18）停留时间方差
genre19: mac在时区4（18-24）停留时间方差
genre20: 每个mac的平均停留时间和总平均停留时间之比
genre21: mac停留时间在5-30分钟之内的次数
genre22: mac在八月第一周停留次数（是否有周期性）
genre23: mac在八月第二周停留次数
genre24: mac在八月第三周停留次数
genre25: mac在八月第四周停留次数"

In [17]:
user_info=pd.read_csv('/Users/apple/Desktop/Data/New feature vector with transaction people/Feature_Vector_with_transaction.csv',index_col="Unnamed: 0")
user_info.head()

,UserId,genre0,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,...,genre16,genre17,genre18,genre19,genre20,genre21,genre22,genre23,genre24,genre25
0,18:59:36:ca:34:e7,1.838739,11.068394,37,32,5,5,14.666667,0.016667,0,...,0.000000,11.872137,0.701905,31.285444,0.224691,7,12,1,3,1
1,48:74:6e:70:88:ab,2.187255,14.326324,17,7,10,4,15.650000,0.016667,4,...,36.745469,0.150802,0.131383,0.040185,0.267279,9,3,1,2,1
2,ac:f7:f3:a7:78:8b,37.183333,0.000000,1,0,1,1,37.183333,37.183333,0,...,0.000000,0.000000,0.000000,0.000000,4.543745,0,0,1,0,0
3,34:6b:d3:df:23:90,1.837879,4.025156,11,8,3,3,7.316667,0.250000,4,...,1.009913,0.000000,11.111111,1.139489,0.224586,4,3,3,0,1
4,60:d9:a0:fc:03:b5,13.533333,477.439630,6,4,2,4,61.900000,0.566667,0,...,0.000000,0.000000,10.969722,940.444444,1.653752,4,2,1,0,0


我把item side information设置成全是0的矩阵（没有item side information）

In [7]:
genres=[[i,0] for i in list(set(ratings['ItemId'].values))]
genres=pd.DataFrame(genres,columns=['ItemId','genre0'])

In [8]:
# Number of latent factors
k=30
k_main=5
k_genre=5
k_demo=5
# Regularization parameter
reg=1e-3
# This time I'll weight the ratings matrix higher
w_main=4
# Fitting the model
# rec3=CMF(k=k, k_main=k_main, k_item=k_genre, k_user=k_demo, w_main=w_main, reg_param=reg)
rec3=CMF(k=k, k_main=k_main, k_item=k_genre, k_user=k_demo, reg_param=reg, w_main=w_main)
rec3.fit(train, genres, user_info, random_seed=32545)
# Making predictions
test['Predicted']=test.apply(lambda x: rec3.predict(x['UserId'],x['ItemId']),axis=1)

INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 215.199005
  Number of iterations: 1001
  Number of functions evaluations: 1092


In [9]:
np.sqrt(np.mean((test.Predicted-test.Rating)**2))

1.7457919837303943

RMSE becomes higher

我先试着删除了波动较大的方差的feature

In [11]:
del user_info['genre12']
del user_info['genre13']
del user_info['genre14']
del user_info['genre15']
del user_info['genre16']
del user_info['genre17']
del user_info['genre18']
del user_info['genre19']

In [12]:
# Number of latent factors
k=30
k_main=5
k_genre=5
k_demo=5
# Regularization parameter
reg=1e-3
# This time I'll weight the ratings matrix higher
w_main=4
# Fitting the model
# rec3=CMF(k=k, k_main=k_main, k_item=k_genre, k_user=k_demo, w_main=w_main, reg_param=reg)
rec3=CMF(k=k, k_main=k_main, k_item=k_genre, k_user=k_demo, reg_param=reg, w_main=w_main)
rec3.fit(train, genres, user_info, random_seed=32545)
# Making predictions
test['Predicted']=test.apply(lambda x: rec3.predict(x['UserId'],x['ItemId']),axis=1)

INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 107.508881
  Number of iterations: 1001
  Number of functions evaluations: 1090


In [13]:
np.sqrt(np.mean((test.Predicted-test.Rating)**2))

1.444269187457658

RMSE becomes smaller, however there is no improvement